In [7]:
!pip install yfinance==0.2.54
!pip install arch

In [14]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from scipy import signal, stats, optimize
from arch import arch_model

In the first three exercises of this exercise sheet, we will use as data the SP500 prices between the 1st of January of 1950 and the 31st of December of 2024, with daily precision.

In [ ]:
# Download the data
df = yf.download('^GSPC',datetime(1950, 1, 1), end=datetime(2024, 12, 31), period="1y", progress=False)['Close']
print(df.head())

YF.download() has changed argument auto_adjust default to True
Ticker      ^GSPC
Date             
1950-01-03  16.66
1950-01-04  16.85
1950-01-05  16.93
1950-01-06  16.98
1950-01-09  17.08


 # Distribution of returns

 As you saw in theory, the time series of returns differs from that of a white noise in several ways.
 - First, its probability distribution is typically leptokurtic, meaning it has fatter tails than a normal distribution, indicating a higher probability of extreme events.
 - Second, it shows a finite (albeit short) autocorrelation time, which means that the power spectrum is not flat.
 - Third, it shows heteroskadisticity in the volatility. Specifically, it experiences volatility clustering, where periods of high volatility tend to follow each other.

 In the first exercise, we will analyze the first aspect, namely the fat tails of the return pdf. A typical distribution to model fat tails is the Pareto distribution, which is a one-sided power law pdf.  
$$
f(x)=\begin{cases}
\frac{\alpha\sigma^\alpha}{(x-\mu)^{\alpha+1}} \quad x>\mu+\sigma \\
0  \quad \quad  x<\mu+\sigma
\end{cases}
$$



# Exercise 1.


1. Compute the log returns of the Close value of the index

2. Plot the Probability Distribution Function of the **absolute value** of the returns and fit the tail to a power law (Pareto distribution).

**Indication:** *the Pareto method from scipy struggles a lot when finding the lower threshold for $x$. The right way to do this exercise is using specialized packages like powerlaw, which automatically find the optimal value of $x_{min}$. I have computed this value for you, so you can stick to scipy: $x_{min} = 0.0209$. Thus, please* **filter out of the dataframe all the values of the absolute log-returns that are smaller than 0.02**.

**Clue:** *Exploit the techniques learnt in the previous assignment (the Pareto random variable is implemented as scipy.stats.pareto)*.

3. Which value do you obtain for the power law exponent $\alpha$?

4. Is the second moment (the variance) finite?

5. Repeat the analysis analysing separately the positive returns and the negative returns (take the absolute value of the latter). Is there any difference when segregating by sign?



# Power Spectrum of financial time-series

The power spectrum of a time series indicates the intensity of the signal for each frequency $\omega$, it can be computed as the Fourier transform of the time-series' autocovariance, $C(s)$,

\begin{equation}
P(\omega)=\int_{-\infty}^{\infty} C(s) e^{-i \omega s} d s
\end{equation}

In practice, it can be easily computed using the **scipy.signal.periodogram** function (https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.periodogram.html).

In the lectures you have seen that the power spectrum of financial series is well described by the functional form

\begin{equation}
P(\omega)\sim \omega^{-2}
\end{equation}



# Exercise 2

Compute the power spectrum of the SP500 prices. Plot it in log-log scale and compare it with the theoretical prediction. Interpret what the exponent of the power spectrum tells us about the nature of the stochastic process.


**Clue:** *recall the `curve_fit` function we used in the previous assignment. Also, keep in mind that we want to fit the tail of the distribution, so it may be useful to filter out the low frequencies from the fit.*

# Moving average and volatility

The moving average and moving volatility are mesures of these quantities over fixed intervals, which are usually named the "window" over which the measures are taken. These can be useful for smoothing the data for visual analysis or even for quantitative analysis. The moving average and moving volatility can be defined as follows

* Moving average

\begin{equation}
\mu_M(j)=\frac{1}{M}\sum_{i=j-M+1}^jX(i)
\end{equation}

* Moving volatility

\begin{equation}
\sigma^2_M(j)=\frac{1}{M-1}\sum_{i=j-M+1}^j(X(i)-\mu_M(j))^2
\end{equation}

Indeed, we can define the moving measure for any observable $\mathcal{L}$ as

\begin{equation}
\mathcal{L}_M(j)\left\{X\right\}=\sum_{i=j-M+1}^j\mathcal{L}\left\{X(i)\right\}
\end{equation}

 # Exercise 3

1. Compute the moving average of the  <u>Close price</u> for the previous low frequency data with different window sizes (e.g. 30, 252, 504). Plot it together with the original time series.

**Clue:** *The `pandas.Series.rolling(window).statistic()` method computes the given statistic over the given window size through all the series.*

*Example: `df["Col1"].rolling(100).mean()`*

2. Compute the moving volatility of the <u>log-returns</u> with different window sizes.

3. Compute the spectrogram of the moving volatility of the log-returns for a window of 30 days. Can you conclude something from the spectrogram?


# The GARCH model

**Note:** for this part, we will use more recent data. Please make sure to update the returns dataset by running the following cell:

In [ ]:
start = datetime(2000, 1, 1)
end = datetime(2023, 12, 31)
prices = yf.download('^GSPC',start = start, end=end, period="1y", progress=False)['Close']
returns = np.log(1+prices.pct_change()).dropna()*100  #We have to remove the NaNs for the model to work and we rescale the data for better predictions (otherwise a warning shows up)

The GARCH model is a fundamental tool in time series analysis for modeling financial data with *heteroskedasticity*, which means that the variability of the process changes over time. Many financial time series, such as stock returns, exhibit periods of high and low volatility, and models like GARCH help us describe this behavior.

GARCH stands for *Generalized Autoregressive Conditional Heteroskedasticity*. Breaking this down:  
- **Autoregressive** means that past values influence present ones.  
- **Conditional** means that today's volatility depends on past information.  
- **Heteroskedasticity** means that the variance changes over time instead of being constant.  

The simplest version of this family is the ARCH(p) model, where the variance of the stochastic process is modeled as a function of past squared values:

$$
\sigma_j^2 = \alpha_0 + \alpha_1 x_{j-1}^2 + \dots + \alpha_p x_{j-p}^2
$$

However, ARCH models often require a large number of terms ($ p $) to capture volatility patterns effectively. To improve this, the GARCH(p,q) model introduces an additional autoregressive component in the variance equation:

$$
\sigma_j^2 = \alpha_0 + \sum_{i=1}^{p} \alpha_i x_{j-i}^2 + \sum_{k=1}^{q} \beta_k \sigma_{j-k}^2
$$


Here, past volatility values $ \sigma_{j-k}^2 $ also contribute to future volatility, making GARCH models more efficient and widely used in finance. Note that GARCH(p,0) is the same as ARCH(p).


To determine the appropriate values of $ p $ and $q $, we use the **Bayesian Information Criterion (BIC)**, which helps us balance model fit and complexity, avoiding overfitting.

GARCH models are implemented in Python using the `arch` library. These models are typically stored in objects that must be fitted to data before making predictions. In the assignment, you will use Python to fit a GARCH model to financial data and analyze its performance. Let us see an example of how to fit a GARCH process with $p=4, q=2$.

*Terminology alert: the function is called arch, but it actually fits data to arbitrary GARCH models*

In [ ]:
# The parameter o is used to isolate the effect of negative returns (asymetric innovation), here we always stick to o=0

arch_object = arch_model(returns, p=4, o=0, q=2)  # Create a python object with our model

fitted_arch = arch_object.fit(disp= False)   # Fit our model by calling the fit method in our class. disp = False removes displaying information about the fitting process.

fitted_arch.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                  ^GSPC   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8350.47
Distribution:                  Normal   AIC:                           16716.9
Method:            Maximum Likelihood   BIC:                           16770.6
                                        No. Observations:                 6036
Date:                Tue, Mar 11 2025   Df Residuals:                     6035
Time:                        12:00:49   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0604  1.049e-02      5.753  8.743e-09 [3.981e-02,8.095e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0462  1.299e-02      3.556  3.765e-04  [2.073e-02,7.166e-02]
alpha[1]       0.0830  2.158e-02      3.844  1.210e-04    [4.066e-02,  0.125]
alpha[2]       0.1335  2.253e-02      5.926  3.106e-09    [8.935e-02,  0.178]
alpha[3]       0.0177  2.855e-02      0.619      0.536 [-3.828e-02,7.364e-02]
alpha[4]   1.7039e-16  3.046e-02  5.594e-15      1.000 [-5.970e-02,5.970e-02]
beta[1]        0.0968      0.101      0.958      0.338      [ -0.101,  0.295]
beta[2]        0.6361  9.662e-02      6.583  4.612e-11      [  0.447,  0.825]
=============================================================================

Covariance estimator: robust
"""

As you can see, we get a lot of nice information about how the parameters were fitted. Apart from the values of the four $\alpha_i$, the two $\beta_i$, and $\omega$, we also get their standard error, their t statistic, their p-value, and their confidence interval.  Moreover, we get the model R^2, log-likelihood, BIC, etc. All these variables are also stored within fitted_arch. For example, fitted_arch.bic returns the BIC.

In [ ]:
fitted_arch.bic

16770.59206187919

(**Optional**: *read about the t-statistic and the p-value. What is the relation between the t-statistic and the t distribution we used in the last assignment? In this example, what conclusions can we get from the values we obtained for t and P>|t|?)*

# Exercise 4

**1. Determine the values of $p$ and $q$ that optimally fit the data to a GARCH process**

Fit the model using different combinations of p and q ($p$ ranging from 1 to 5, $q$ ranging from 0 to 4) and compute their BIC values to determine the optimal value for $p$ and $q$.

**2. Forecast using optimal model**

Use the optimal model that you obtained to perform a forecast of the volatility one step into the future. Specifically, for every time $t$, forecast the volatility at time $t+1$. Plot the predictions along with the moving volatility with window $T=30$. Do you think the model is performing well?

*Indication: the forecast is performed automatically using the forecast method of the fitted model object. The syntax of this method is `forecasts = fitted_arch.forecast(horizon=1, start=0, reindex=True, align=target)`.If you are curious on what the arguments of this function are, check it out at https://arch.readthedocs.io/en/latest/. To extract the variance of the forecast, simply call 'forecasts.variance'.*




**3 (Optional). Simulate five realizations of future price dynamics for one year ($N=252$) using the optimal GARCH model**. To simulate future values of the time series, you should call `arch_model.simulate(params_garch, N_steps)`, with `params_garch = fitted_arch.params`. Recall that the GARCH model simulates returns, which have to be converted back to prices. Compare with the real evolution of the SP500 in 2024.

*Indication: remember that we scaled the returns in this section by a factor of 100. To compare with real data, we must divide the simulated returns by a factor 100 to cancel out the rescaling.*